In [6]:
val PATH = "file:///Users/lzz/work/SparkML/"

# Spark构建聚类模型
聚类算法有很多种，MLlib 目前提供了 K-means 聚类算法，该算法将一系列样本分割成 K 个不同的类族（其中 K 是模型的输入参数），目的是最小化所有类族中的方差之和，其形式化的目标函数称为类族内的方差和（within cluster sum of squared errors, WCSS）
## 本章节分为以下几个部分
* 一、从数据中提取正确的特征
* 二、训练聚类模型
* 三、使用聚类模型进行预测
* 四、评估聚类模型的性能
* 五、聚类模型参数调优

## 一、从数据中提取正确的特征
类似大多数机器学习模型，K－均值聚类需要数值向量作为输入，于是用于分类和回归的特征提取和变换方法也适用于聚类。  
K-均值和最小方差回归一样使用方差函数作为优化目标，因此容易受到离群值（outlier）和较大方差的特征影响。  
对于回归和分类问题来说，上述问题可以通过特征的归一化和标准化来解决，同时可能有助于提升性能。但是某些情况我们可能不希望数据被标准化，比如根据某个特定的特征找到对应的类族。
### 从MovieLens 数据集提取特征
我们使用ml-100k电影评分数据集，第一个是电影的打分的数据集（u.data）,第二个是用户数据（u.user）,第三个是电影数据（u.item）。除此之外，我们从题材文件中获取来每个电影的题材（u.genre）。  
输出电影数据集的首行：

In [7]:
val movies = sc.textFile( PATH+"data/ml-100k/u.item")
println( movies.first )

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0


#### 1、提取电影的题材标签
在进一步处理之前，我们先从u.genre文件中提取题材的映射关系。根据之前对数据集的输出结果来看，需要将题材的数字编号映射到可读的文字版本。查看u.genre开始几行数据：

In [8]:
val genres = sc.textFile( PATH + "data/ml-100k/u.genre")
genres.take(5).foreach(println)

unknown|0
Action|1
Adventure|2
Animation|3
Children's|4


上面输出的数字表示相关题材的索引，比如0是unknown的索引。索引对应来每部电影关于题材的特征二值子向量（既前面数据中0和1）。  
为例提取题材的映射关系，我们对每一行数据进行分割，得到具体的<题材，索引>键值对。注意处理过程中需要处理最后的空行，不然会抛出异常

In [16]:
val genreMap = genres.filter( !_.isEmpty ).map( line => line.split("\\|")).
map(array => (array(1),array(0))).collectAsMap
println(genreMap)

Map(2 -> Adventure, 5 -> Comedy, 12 -> Musical, 15 -> Sci-Fi, 8 -> Drama, 18 -> Western, 7 -> Documentary, 17 -> War, 1 -> Action, 4 -> Children's, 11 -> Horror, 14 -> Romance, 6 -> Crime, 0 -> unknown, 9 -> Fantasy, 16 -> Thriller, 3 -> Animation, 10 -> Film-Noir, 13 -> Mystery)


接下来，我们需要为电影数据和题材映射关系创建新的RDD，其中包含电影ID，标题和题材。当我们用聚类模型评估每个电影的类别时，可以哟过生成的RDD得到可读的输出。  
接下来，我们对每部电影提取相应的题材（是String形式而不是Int 索引）。使用zipWithIndex方法统计包含题材索引的集合，这样就能将集合中的索引映射到对应的文本信息。最后，输出RDD第一条记录：  

In [17]:
val titlesAndGenres = movies.map(_.split("\\|")).map{ array =>
    val genres = array.toSeq.slice(5, array.size)
    val genresAssigned = genres.zipWithIndex.filter{ 
        case (g, idx) =>
            g == "1"
    }.map{
        case (g, idx) => genreMap(idx.toString)
    }
    (array(0).toInt, (array(1), genresAssigned))
}
println(titlesAndGenres.first)

(1,(Toy Story (1995),ArrayBuffer(Animation, Children's, Comedy)))


#### 2、训练推荐模型
要获取用户和电影因素向量，首先需要训练一个新的推荐模型。

In [20]:
import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.Rating
val rawData = sc.textFile( PATH + "data/ml-100k/u.data" )
val rawRatings = rawData.map( _.split("\t").take(3) )
val ratings = rawRatings.map{ case Array(user, movie, rating) =>
    Rating(user.toInt, movie.toInt, rating.toDouble)
}
ratings.cache
val alsModel = ALS.train(ratings, 50, 10, 0.1)

最小二乘法（Alternating Least Squares,ALS）模型返回例两个键值RDD（user－Features和productFeatures）。这两个RDD的键为用户ID或者电影ID，值为相关因素。我们还需要提取相关的因素并转化到MLlib的Vector中作为聚类模型的训练输入。

In [ ]:
import org.apache.spark.mllib.linalg.Vectors
val movieFactors = alsModel.productFeatures.map{ 
    case(id, factor) => (id, Vectors.dense(factor) )
}
val movieVectors = movieFactors.map( _._2 )
val userFactors = alsModel.userFeatures.map {
    case( id, factor ) => (id, Vectors.dense(factor) )
}
val userVectors = userFactors.map( _._2 )

#### 3、归一化
观察输入数据的相关因素特征向量的分布，这可以分析出是否需要对训练模型进行归一化，具体做法是用MLlib 的RowMatrix进行各种统计

In [23]:
import org.apache.spark.mllib.linalg.distributed.RowMatrix
val movieMatrix = new RowMatrix( movieVectors )
val movieMatrixSummary = movieMatrix.computeColumnSummaryStatistics()
val userMatrix = new RowMatrix(userVectors)
val userMatrixSummary = userMatrix.computeColumnSummaryStatistics()
println("Movie factors mean: " + movieMatrixSummary.mean )
println( "Movie factors variance: " + movieMatrixSummary.variance )
println( "User factors mean: " + userMatrixSummary.mean )
println( "User factors variance: " + userMatrixSummary.variance )

Movie factors mean: [-0.010956087400287078,-0.3503618170650178,0.06974321778051942,-0.1316383504373148,-0.3175018092607278,-0.1322647106693895,0.2537778433328245,-0.2029801309890879,-0.12961386425209565,-0.07082069156194899,0.06205439440201192,0.29001069000651375,-0.18597369739464903,-0.23937735319862308,-0.14342491847545685,0.18101149542654485,-0.13804024072643173,0.2850367666048985,0.12990755598051892,-0.2940282667527131,0.10936497810207288,-0.056700820289192294,0.008779148148335511,-0.09952976911271119,-0.12124526842150213,-0.1047346330659357,-0.01800057373980534,0.20805219400947975,-0.025770092278983504,-0.16276854660068052,0.14352873442556296,-0.04190886803719547,0.4564002303815088,-0.13991199983838468,0.24892268343949558,0.03683881854085137,-0.12161360237334293,0.05101734977006077,-0.1716588720612823,-0.25528384724181186,-0.24128689111316304,0.3161922110050902,-0.37518932943231786,-0.07739395342787778,-0.033145745157435264,-0.3182618436521061,0.31724803911912525,-0.29054483322329

## 二、训练聚类模型
在Mllib中训练K-均值的方法和其他模型类似，只要把包含训练数据的RDD传入KMeans对象的train方法即可。注意，因为聚类不需要标签，所以不用LabeldPoint实例，而是使用特征向量接口，既RDD的Vector数组即可。
### 用MovieLens数据集训练聚类模型
MLlib的K-均值提供了随机和K-means||两种初始化方法，后者是默认的初始化。因为两种方法都试随机选择，所以每次模型训练的结果都不一样。  
K-均值通常不能收敛到全局最优解，所以实际应用中需要多次训练并选择最优模型。MLlib提供了完成多次模型训练的方法。经过损失函数的评估，将性能最好的一次训练选定为最终的模型。  
设置模型参数：K（numClusters）,最大迭代次数（numIteration）,和训练次数（numRuns）:

In [24]:
import org.apache.spark.mllib.clustering.KMeans
val numClusters = 5
val numiterations = 10
val numRuns = 3
val movieClusterModel = KMeans.train( movieVectors, numClusters, numiterations, numRuns)

In [25]:
val movieClusterModelConverged = KMeans.train( movieVectors, numClusters, 100)

In [26]:
val userClusterModel = KMeans.train( userVectors, numClusters, numiterations, numRuns)

## 三、使用聚类模型进行预测
使用训练的K-means模型进行预测，下面对单独对样本进行预测：

In [27]:
val movie1 = movieVectors.first
val movieCluster = movieClusterModel.predict(movie1)
println( movieCluster )

3


也可以通过传入一个RDD[Vector]数组对多个输入样本进行预测：

In [28]:
val predictions = movieClusterModel.predict( movieVectors )
println( predictions.take(10).mkString(",") )

3,1,0,1,0,1,1,0,0,1


### 用MovieLens数据集解释类别预测
前面我们已经介绍了如何对一系列输入数据进行预测，但是如何对预测结果进行评估呢？
#### 解释电影类族
因为K-means最小化对目标函数是样本到其类中心对欧拉距离之和，我们便可以将“最靠近类中心”定义为最小的欧拉距离。下面我们定义这个度量函数，注意引入Breeze 库（MLlib的一个依赖库）用于线性代数和向量运算：

In [30]:
import breeze.linalg._
import breeze.numerics.pow
def computeDistance( v1: DenseVector[Double], v2: DenseVector[Double] ) = pow( v1 - v2, 2 ).sum

下面我们利用上面的函数对每个电影计算其特征向量与所属类族中心向量的距离。为了让结果具有可读性，输出结果中添加了电影的标题和题材数据：

In [31]:
val titlesWithFactors = titlesAndGenres.join(movieFactors)
val moviesAssigned = titlesWithFactors.map{
    case ( id, ((title, genres), vector)) =>
        val pred = movieClusterModel.predict( vector )
        val clusterCentre = movieClusterModel.clusterCenters( pred )
        val dist = computeDistance( DenseVector( clusterCentre.toArray), DenseVector(vector.toArray) )
            (id, title, genres.mkString(" "), pred, dist)
}
val clusterAssignments = moviesAssigned.groupBy{
    case (id, title, genres, cluster, dist) => cluster
}.collectAsMap

我们得到一个RDD，其中每个元素是关于某个类族的键值对，健是类族的标识，值是若干电影和相关信息组成的集合。电影的信息为：电影ID，标题，题材，类别索引，以及电影的特征向量和类中心的距离。

In [33]:
for( (k, v) <- clusterAssignments.toSeq.sortBy(_._1) ){
    println( s"Cluster $k:" )
    val m = v.toSeq.sortBy( _._5 )
    println( m.take(20).map{ 
        case( _, title, genres, _, d) => (title, genres, d)
    }.mkString("\n"))
    println("---------\n")
}

Cluster 0:
(King of the Hill (1993),Drama,0.16150193866972645)
(Witness (1985),Drama Romance Thriller,0.2274559199018781)
(All Over Me (1997),Drama,0.2629440718963916)
(Scream of Stone (Schrei aus Stein) (1991),Drama,0.26750415112992426)
(Ed's Next Move (1996),Comedy,0.33834019947892024)
(I Can't Sleep (J'ai pas sommeil) (1994),Drama Thriller,0.34065424343733147)
(Wings of Courage (1995),Adventure Romance,0.3574027323494375)
(Silence of the Palace, The (Saimt el Qusur) (1994),Drama,0.3667946097617337)
(Land and Freedom (Tierra y libertad) (1995),War,0.3667946097617337)
(Normal Life (1996),Crime Drama,0.3667946097617337)
(Eighth Day, The (1996),Drama,0.3667946097617337)
(Two Friends (1986) ,Drama,0.3667946097617337)
(Dadetown (1995),Documentary,0.3667946097617337)
(Girls Town (1996),Drama,0.3667946097617337)
(Big One, The (1997),Comedy Documentary,0.3667946097617337)
(Hana-bi (1997),Comedy Crime Drama,0.3667946097617337)
(� k�ldum klaka (Cold Fever) (1994),Comedy Drama,0.366794609761733

## 四、评估模型的性能
聚类的评估通常分为两个部分：内部评估和外部评估。内部评估表示评估过程使用训练模型时使用的训练数据，外部评估则使用训练数据之外的数据。  
### 内部评估指标
通常的内部评价指标包括WCSS，Davies－Bouldin指数、Dunns 指数和轮廓系数（silhouette coefficient）。所有这些度量指标都是使用聚类内部的样本距离尽可能接近，不同类族的样本相对较远。
### 外部评价指标
因为聚类被认为是无监督分类，如果有一些带标注的数据，便可以用这些标签来评估聚类模型。可以使用聚类模型预测类族（类标签），使用分类模型中类似的方法评估预测值和真实标签的误差（既真假阳性率和真假阴性率）。  
具体方法包括Rand measure、 F-measure、雅卡尔系数（Jaccard index）等。
### 在MovieLens 数据集计算性能
MLlib提供的函数computeCost可以方便的计算出给定输入数据的RDD[Vector]的WCSS。下面我们使用这个方法计算电影和用户训练数据的性能：

In [34]:
val movieCost = movieClusterModel.computeCost( movieVectors )
val userCost = userClusterModel.computeCost( userVectors )
println( "WCSS for movies: " + movieCost )
println( "WCSS for movies: " + userCost )

WCSS for movies: 2282.7384462397395
WCSS for movies: 1476.204833283481


## 五、聚类模型参数调优
### 通过交叉验证选择K
我们按6:4将数据集分割为训练集和测试集，然后在训练集上训练模型，在测试集上评估感兴趣的指标的性能

In [36]:
val trainTestSplitMovies = movieVectors.randomSplit( Array(0.6, 0.4), 123 )
val trainMovies = trainTestSplitMovies(0)
val testMovies = trainTestSplitMovies(1)
val costsMovies = Seq(2 ,3 ,4 ,5 ,10 ,20).map{
    k => (k, KMeans.train(trainMovies, numiterations, k, numRuns).computeCost(testMovies) )
}
println( "Movie clustering cross-validation:")
costsMovies.foreach{ case( k, cost) => println(f"WCSS for K=$k id $cost%2.2f") }

Movie clustering cross-validation:
WCSS for K=2 id 884.89
WCSS for K=3 id 867.80
WCSS for K=4 id 872.01
WCSS for K=5 id 879.41
WCSS for K=10 id 863.81
WCSS for K=20 id 860.85


从结果可以看出，随着类中心数目增加，WCSS值会出现下降，然后又开始增大。另外一个现象，K-means在交叉验证的情况，WCSS随着K的增大持续减小，但是达到某个值后，下降的速度突然会变得很平缓。这时的K通常为最优的K值（拐点）。   
根据预测结果，我们选择最优的K＝10.需要说明是，模型计算的类族需要人工解释。尽管较大的K值从数学的角度可以得到更优的解，但是类族太多就会变得难以理解和解释。  
为了实验的完整性，我们还计算了用户聚类在交叉验证下的性能：

In [38]:
val trainTestSplitUsers = userVectors.randomSplit( Array(0.6, 0.4), 123)
val trainUsers = trainTestSplitUsers(0)
val testUsers = trainTestSplitUsers(1)
val costsUsers = Seq( 2, 3, 4, 5 , 10, 20).map{
    k => (k, KMeans.train(trainUsers, numiterations, k, numRuns).computeCost(testUsers))
}
println( "User clustering cross-validation:")
costsUsers.foreach{
    case(k, cost) => println( f"WCSS for K=$k id $cost%2.2f")
}

User clustering cross-validation:
WCSS for K=2 id 591.78
WCSS for K=3 id 594.46
WCSS for K=4 id 598.01
WCSS for K=5 id 594.43
WCSS for K=10 id 596.87
WCSS for K=20 id 594.06
